In [1]:
from transformers import ViTMAEConfig, ViTMAEModel, ViTMAEForPreTraining
%load_ext autoreload


# Initializing a ViT MAE vit-mae-base style co`nfiguration
configuration = ViTMAEConfig(
    image_size=100,
    num_channels=1,
    hidden_size=480,
    intermediate_size=1024,
    decoder_intermediate_size=1024,
    patch_size=10,
    mask_ratio=0,
)

# Initializing a model (with random weights) from the vit-mae-base style configuration
model = ViTMAEForPreTraining(configuration)

# Accessing the model configuration
configuration = model.config

print('number of parameters: ', sum(p.numel() for p in model.parameters()))


number of parameters:  40191300


In [ ]:
from dataloader import BATCH_SIZE, square_xrd_dataloader, square_binary_dataloader

import torch
from torch import nn, optim
mse_loss = nn.MSELoss()

def train_model(num_epochs=100):
    outputs = []
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    for epoch in range(num_epochs):
        for idx, data in enumerate(square_binary_dataloader):
            # ===================forward=====================
            output = model(data)
            loss = mse_loss(output.logits, data.squeeze())
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            if idx % 5 == 0:
                print(f"Finished batch {idx} in epoch {epoch + 1}. Loss: {loss.item():.4f}")

        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
        outputs.append((epoch, data, output))



# Train the model

model.train(True)
train_model(num_epochs=1)
model.train(False)

Finished batch 0 in epoch 1. Loss: 1.1763


In [13]:
# this is a demo for testing

from transformers import AutoFeatureExtractor, ViTMAEForPreTraining
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/vit-mae-base")
model_pretrained = ViTMAEForPreTraining.from_pretrained("facebook/vit-mae-base")

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model_pretrained(**inputs)
loss = outputs.loss
mask = outputs.mask
ids_restore = outputs.ids_restore

inputs['pixel_values'].type()